In [1]:
from antifraud import read_adj_dict_from_file, adjacency_dict
from graph_algorithms import Graph

# Read files

In [3]:
%%time

batch  = '../paymo_input/batch_payment.txt'
stream = '../paymo_input/stream_payment.txt'

adj_list   = read_adj_dict_from_file (batch)
#adj_stream = read_adj_dict_from_file (stream)

CPU times: user 9.19 s, sys: 208 ms, total: 9.4 s
Wall time: 9.4 s


In [4]:
batch_graph = Graph (adj_list.copy ())

# Dynamic programming
### Idea
The adjacency list 'adj_list' is that of the graph of 1 degree of separation. Using just this to calculate the distance between 2 arbitrary node turns out to be too computationally expensive (for my taste at least) when dealing with the stream_processing.txt file.

So, my current idea is to build adjacency lists of 2, 3, and 4 degrees of separation. That should eliminate a good deal of redundant computations and hopefully the result will be noticeably faster.

### Test Problem
<img src="image/example_graph.jpg" width="50%">
Let the graph G be defined by the following list of edges:
$$
\text{edges} (G) =
[(1, 13), (1, 3), (1, 4),
 (2, 12),
 (4, 13),
 (3, 5),
 (5, 6), (5, 11),
 (6, 8), (6, 9), (6, 10),
 (7, 8),
 (10, 11)]
$$

In [2]:
# exclusive 1st order adjacency list for G (does not include nodes separated by degree zero)
adj_1_exclusive = {1 : {3, 4, 13}, 2 : {12}, 3 : {1, 5}, 4 : {1, 13}, 5 : {3, 6, 11}, 6 : {5, 8, 9, 10},
7 : {8, 12}, 8 : {6, 7}, 9 : {6}, 10: {6, 11}, 11: {5, 10}, 12: {2, 7}, 13: {1, 4}}

# inclusive 1st order adjacency list for G
adj_1 = {}
for key in adj_1_exclusive.keys ():
    adj_1 [key] = {key} | adj_1_exclusive [key]

# exclusive 2nd order adjacency list for G
adj_2_exclusive = {1 : {5}, 2 : {7}, 3 : {4, 6, 11, 13}, 4 : {3}, 5 : {1, 8, 9, 10}, 6 : {3, 7, 11},
7 : {2, 6}, 8 : {5, 9, 10, 12}, 9 : {5, 8, 10}, 10: {5, 8, 9}, 11: {3, 6}, 12: {8}, 13: {3}}

# inclusive 2nd order adjacency list for G
adj_2 = {}
for key in adj_1.keys ():
    adj_2 [key] = adj_1 [key] | adj_2_exclusive [key]

In [3]:
edges = [(1, 13), (1, 3), (1, 4), (2, 12), (4, 13), (3, 5), (5, 6),
         (5, 11), (6, 8), (6, 9), (6, 10), (7, 8), (7, 12), (10, 11)]
G = Graph ({})

# create graph G from list of edges
G.add_edges (edges)

G

{1: {3, 4, 13}, 13: {1, 4}, 3: {1, 5}, 4: {1, 13}, 2: {12}, 12: {2, 7}, 5: {11, 3, 6}, 6: {8, 9, 10, 5}, 11: {10, 5}, 8: {6, 7}, 9: {6}, 10: {11, 6}, 7: {8, 12}}

In [9]:
# build 1st order inclusive adjaceny list
G.build_adj ()
# build higher order inclusive adjaceny list
G.build_higher_order_adj_lists ()

In [10]:
# compare inclusive adjacency lists or order 1 & 2
print (G.adj_1 == adj_1)
print (G.adj_2 == adj_2)

True
True


In [11]:
# Check for self-consistency
G.is_self_consistent ()

True

# Tests and examples

list of tests and excamples:
1. test case for function adjacency_dict
2. Confirm algo can handle being given node not in graph
3. Check how long it takes to confirm 2 nodes are separated by degree 4 (**67ms**!)
4. Check how long it takes to completely traverse the graph given in batch_processing.txt    (**< 200ms!**)

### test case for function adjacency_dict
```
a
| \
b--c--d--e
```
connections: (a, b), (a, c), (b, c), (c, d), (d, e)

or, as 2 lists: (a, a, b, c, d) and (b, c, c, d, e)

In [12]:
def pass_fail (logic, message = ''):
    if logic:
        print ('[PASS]: ', end = '')
    else:
        print ('[FAIL]: ', end = '')
    print (message)
    

In [13]:
ex_adj = adjacency_dict (['a', 'a', 'b', 'c', 'd'],
                         ['b', 'c', 'c', 'd', 'e'])
ex_graph = Graph (ex_adj)

pass_fail (ex_graph.distance ('a', 'e') == 3, 'method distance')
pass_fail (ex_graph.degree_list ('c') == [{'c'}, {'a', 'b', 'd'}, {'e'}],
           'method degree_list')

# Add edge w/ new node 'f'
ex_graph.add_edge (('f', 'e'))

pass_fail (ex_graph.length == 6,
           'length update after adding new edge')
pass_fail (ex_graph.graph == {'a': {'c', 'b'}, 'b': {'c', 'a'}, 'c': {'d', 'a', 'b'}, 'd': {'c', 'e'}, 'e': {'f', 'd'}, 'f': {'e'}},
           'graph update after adding new edge')

[PASS]: method distance
[PASS]: method degree_list
[PASS]: length update after adding new edge
[PASS]: graph update after adding new edge


In [14]:
adjacency_dict (['a', 'a', 'b', 'c', 'd'],
                ['b', 'c', 'c', 'd', 'e']) == {'a': {'b', 'c'},
                                               'b': {'a', 'c'},
                                               'c': {'a', 'b', 'd'},
                                               'd': {'c', 'e'},
                                               'e': {'d'}}

True

### Confirm algo can handle being given node not in graph

In [16]:
%%time

batch_graph.distance (49466, 27060006)

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 23.1 µs


-1

### Check how long it takes to confirm 2 nodes are separated by degree 4

In [17]:
%%time

batch_graph.distance (49466, 2706)

CPU times: user 51.4 ms, sys: 4.04 ms, total: 55.4 ms
Wall time: 56 ms


4

### Check how long it takes to completely traverse the graph given in batch_processing.txt
As well as calculate and print some interesting statistics regarding the number of nodes in the graph separated by a certain degree

In [18]:
%%time
# takes <200ms 

deg = batch_graph.degree_list (49466, upto = -1) # the upto = -1 tells it to traverse entire graph

for k in range (len (deg)):
    print ('deg: {:2}, #nodes: {}'.format (k, len (deg [k])))
print ('\n')

deg:  0, #nodes: 1
deg:  1, #nodes: 1
deg:  2, #nodes: 37
deg:  3, #nodes: 4501
deg:  4, #nodes: 42831
deg:  5, #nodes: 27365
deg:  6, #nodes: 2400
deg:  7, #nodes: 208
deg:  8, #nodes: 14
deg:  9, #nodes: 2


CPU times: user 152 ms, sys: 8.02 ms, total: 160 ms
Wall time: 161 ms
